<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homeworks 8 & 9 (Combined): Practical - Faces <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Amirhossein Ramazani Bonab - Yalda Shabanzadeh
    
____

### Full Name : Mehdi Lotfian
### Student Number : 99105689
### Colab Link: https://colab.research.google.com/drive/1y8Nbgfb1k0AdyDWuij8Yu71U-UNp3V0F?usp=sharing
___

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file called `images.zip`, and the `meta.csv` determines from which person each image was taken.

In [156]:
!wget -O images.zip https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
!wget -O meta.csv https://www.dropbox.com/s/vqt1v8sfmz18rcf/faces-metadata.csv?dl=1

--2023-06-27 20:22:16--  https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/wrias0bjyte3rr2/images.zip [following]
--2023-06-27 20:22:16--  https://www.dropbox.com/s/dl/wrias0bjyte3rr2/images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4e7f5447f577fa7ca6bab01ee9.dl.dropboxusercontent.com/cd/0/get/B-wWPpgxVpEhXHmsjQaM7QjeHF1pgsN3OEYh2aipKbA4JQs41Aj-bVjqLgLrFxIqC4gXdly3uq0VCuy82gnMjhgLchl_KANN70JykqE2KWKyvUfwEhzUMqEIs7TKtewVALBUsG-fBbecslXj-bCIdXNY3CiQEn0kgqP7c72Rd7WEp9FkS3kWaGkpvyC-gbu5Hy0/file?dl=1# [following]
--2023-06-27 20:22:17--  https://uc4e7f5447f577fa7ca6bab01ee9.dl.dropboxusercontent.com/cd/0/get/B-wWPpgxVpEhXHmsjQaM7QjeHF1pgsN3OEYh2aipKbA4JQs41Aj-bVjqLgLrFxI

In [157]:
# you can use the following packages
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px

# Sec 1: Data Preparation

Load all images as a numpy array (called `x`) and the metadata as a dataframe **(P1-1: 25 points)**

In [158]:
from zipfile import ZipFile

In [159]:
file_names = ['content/img-data/' + str(i) + '.jpg' for i in range(400)]
x = []
for file_name in file_names:
    with ZipFile( "images.zip", "r") as zip_ref:
        with zip_ref.open(file_name) as image_file:
            img = np.array(Image.open(image_file))
            x.append(img)
x = np.array(x)
x

array([[[ 82,  96, 108, ...,  97,  87,  81],
        [ 86, 100, 112, ...,  96,  84,  76],
        [ 88, 101, 116, ...,  97,  81,  74],
        ...,
        [ 51,  49,  54, ...,  42,  41,  42],
        [ 52,  50,  55, ...,  40,  39,  40],
        [ 53,  51,  54, ...,  38,  37,  37]],

       [[113, 121, 133, ...,  49,  52,  44],
        [119, 123, 132, ...,  51,  55,  49],
        [127, 128, 135, ...,  50,  54,  52],
        ...,
        [197, 198, 199, ...,  37,  37,  37],
        [196, 198, 200, ...,  33,  33,  33],
        [197, 199, 202, ...,  38,  38,  38]],

       [[ 80, 105, 125, ..., 108,  92,  79],
        [ 75,  98, 123, ..., 107,  91,  79],
        [ 66,  88, 121, ..., 105,  90,  79],
        ...,
        [ 34,  18,  26, ...,  35,  35,  38],
        [ 32,  17,  25, ...,  35,  36,  38],
        [ 32,  17,  24, ...,  36,  36,  39]],

       ...,

       [[124, 138, 154, ...,  80,  57,  55],
        [122, 137, 156, ...,  76,  52,  48],
        [121, 137, 156, ...,  71,  50,  44

In [160]:
df = pd.read_csv('meta.csv')
df

,path,label
0,0.jpg,jones
1,1.jpg,jones
2,2.jpg,jones
3,3.jpg,jones
4,4.jpg,jones
...,...,...
395,395.jpg,vaux
396,396.jpg,vaux
397,397.jpg,vaux
398,398.jpg,vaux


Flatten and normalize the images **(P1-2: 10 points)**

Note: For this part, normalization only means dividing by 255.

In [161]:
x = x.reshape(x.shape[0], -1)
x = x / 255.0
x

array([[0.32156863, 0.37647059, 0.42352941, ..., 0.14901961, 0.14509804,
        0.14509804],
       [0.44313725, 0.4745098 , 0.52156863, ..., 0.14901961, 0.14901961,
        0.14901961],
       [0.31372549, 0.41176471, 0.49019608, ..., 0.14117647, 0.14117647,
        0.15294118],
       ...,
       [0.48627451, 0.54117647, 0.60392157, ..., 0.16862745, 0.17254902,
        0.17254902],
       [0.20392157, 0.24313725, 0.20784314, ..., 0.56862745, 0.58039216,
        0.58823529],
       [0.49411765, 0.48627451, 0.26666667, ..., 0.3372549 , 0.35686275,
        0.38823529]])

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y` **(P1-3: 5 points)**

In [162]:
le = LabelEncoder()
y = le.fit_transform(df['label'])
y

array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 33, 33, 33, 33, 33, 33, 33,
       33, 33, 33,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 24,
       24, 24, 24, 24, 24

Split the data into training (80%), validation (10%), and testing (10%) parts. **(P1-4: 10 points)**

Note: The split procedure should maintain the class balance. This is sometimes called "[stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling)".

In [163]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify = y, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.11, stratify = y_train, random_state=1)

In [164]:
unique_values, counts = np.unique(y_train, return_counts=True)
print(unique_values)
print(counts)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38]
[ 8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8 16  8  8  8  8  8  8  8  8]


# Sec 2: SVM

## Subsec 2.1.

Train a SVM using `sklearn`. Note that the hyperparameter tuning should be done using the "validation" set. **(P2-1-1: 20 points)**

In [165]:
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score

In [166]:
C_values = [0.001, 0.01, 0.1, 1, 10]
gamma_values = [0.001, 0.01, 0.1, 1, 10, "scale", "auto"]

best_accuracy = 0.0
best_gamma = 0
best_C = 0
for gamma in gamma_values:
    for C in C_values:
        svm_sklearn = svm.SVC(gamma=gamma, C=C)
        svm_sklearn.fit(x_train, y_train)
        val_predictions = svm_sklearn.predict(x_val)
        val_accuracy = accuracy_score(y_val, val_predictions)
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_gamma = gamma
            best_C = C

In [167]:
print(f"Best Gamma: {best_gamma}, Best C: {best_C}, Best Accuracy: {best_accuracy}")

Best Gamma: 0.001, Best C: 10, Best Accuracy: 0.925


Report the accuracy and F1-score of the classifier on the testing set **(P2-1-2: 10 points)**

In [168]:
svm_sklearn = svm.SVC(gamma=best_gamma, C=best_C)
svm_sklearn.fit(x_train, y_train)
y_pred_svm_sklearn = svm_sklearn.predict(x_test)

In [169]:
print(f"Accuracy: {accuracy_score(y_pred_svm_sklearn, y_test)}, F1 score: {f1_score(y_pred_svm_sklearn, y_test, average='weighted')}")

Accuracy: 0.975, F1 score: 0.985


## Subsec 2.2.

Train a SVM from scratch using the following class **(P2-2-1: 40 points)**

Note: For this part, you are not required to use the validation set for hyperparameter tuning. However, you might get better results if you do that.

In [170]:
class SVM:
    def __init__(self, C = 1.0):
        self.C = C
        self.w = 0
        self.b = 0

    def hingeloss(self, w, b, x, y):
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

            loss = reg + self.C * max(0, 1-opt_term)
        return loss[0][0]

    def fit(self, X, y, batch_size=100, learning_rate=0.001, epochs=1000):
        """Fit the SVM model to dataset X in one-vs-one manner.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        y : array-like of shape (n_samples,)
            Training labels, where `n_samples` is the number of samples.

        Returns
        -------
        self : object
            Returns the instance itself.
        """

        ############ Your Code ############
        # STEPS (For each of the classifiers you would need to train)
        # 1. Setup SVM objective function in QP form
        # 2. Solve the QP problem and get the optimal lagrange multipliers
        # 3. Get support vectors and calculate the weight vector

        number_of_features = X.shape[1]
        number_of_samples = X.shape[0]

        c = self.C
        ids = np.arange(number_of_samples)

        np.random.shuffle(ids)
        w = np.zeros((1, number_of_features))
        b = 0
        losses = []

        for i in range(epochs):
            l = self.hingeloss(w, b, X, y)

            losses.append(l)

            for batch_initial in range(0, number_of_samples, batch_size):
                gradw = 0
                gradb = 0

                for j in range(batch_initial, batch_initial + batch_size):
                    if j < number_of_samples:
                        x = ids[j]
                        ti = y[x] * (np.dot(w, X[x].T) + b)

                        if ti > 1:
                            gradw += 0
                            gradb += 0
                        else:
                            gradw += c * y[x] * X[x]
                            gradb += c * y[x]

                w = w - learning_rate * w + learning_rate * gradw
                b = b + learning_rate * gradb

        self.w = w
        self.b = b

        return self.w, self.b, losses

    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        y_pred : ndarray of shape (n_samples)
            Your predictions for the class of each sample
        """

        ############ Your Code ############
        # Predict using each classifier and get votes for each class
        prediction = np.dot(X, self.w[0]) + self.b
        return np.sign(prediction)

In [171]:
C_values = [0.001, 0.01, 0.1, 1, 10]

best_accuracy_scratch = 0.0
best_C_scratch = 0
for C in C_values:
    svm = SVM(C=C)
    svm.fit(x_train, y_train)
    val_predictions = svm_sklearn.predict(x_val)
    val_accuracy = accuracy_score(y_val, val_predictions)
    if val_accuracy > best_accuracy:
        best_accuracy_scratch = val_accuracy
        best_C_scratch = C

In [172]:
print(f"Best C: {best_C_scratch}, Best Accuracy: {best_accuracy_scratch}")

Best C: 0, Best Accuracy: 0.0


Report the accuracy and F1-score of the classifier on the testing set **(P2-2-2: 10 points)**

In [199]:
svm = SVM(best_C)
svm.fit(x_train, y_train)
y_pred_svm = svm.transform(x_test)

In [200]:
print(f"Accuracy: {accuracy_score(y_pred_svm, y_test)}, F1 score: {f1_score(y_pred_svm, y_test, average='weighted')}")

Accuracy: 0.025, F1 score: 0.04878048780487806


# Sec 3: PCA

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`. **(P3-1: 30 points)**

In [175]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['label'])

target_names = ['jones', 'taylor', 'anderson', 'wilson']
target_indices = label_encoder.transform(target_names)

filtered_images = x[np.isin(y, target_indices)]
filtered_labels = y_encoded[np.isin(y, target_indices)]
filtered_images = filtered_images.reshape(filtered_images.shape[0], -1)

filtered_images.shape

(40, 4096)

In [176]:
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None

    def fit(self, X):
        """Fit the model with X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        ############ Your Code ############

        X_meaned = X - np.mean(X , axis = 0)

        cov_mat = np.cov(X_meaned , rowvar = False)

        eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)

        sorted_index = np.argsort(eigen_values)[::-1]
        sorted_eigenvalue = eigen_values[sorted_index]
        self.components_ = eigen_vectors[:,sorted_index]

        return self

    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        X_new : ndarray of shape (n_samples, n_components)
            Transformed values.
        """
        assert self.components_ is not None

        ############ Your Code ############

        X_meaned = X - np.mean(X , axis = 0)

        eigenvector_subset = self.components_[:,0:self.n_components]

        X_reduced = np.dot(eigenvector_subset.transpose(), X_meaned.transpose()).transpose()

        return X_reduced

In [177]:
pca = PCA(n_components=3)
pca.fit(filtered_images)

In [178]:
pca_result = pca.transform(filtered_images)
pca_result.shape

(40, 3)

In [179]:
df_pca = pd.DataFrame({'PC1': pca_result[:, 0], 'PC2': pca_result[:, 1], 'PC3': pca_result[:, 2], 'Label': filtered_labels})
df_pca

,PC1,PC2,PC3,Label
0,-1.373084,6.218771,1.209337,17
1,5.937131,1.932532,5.918058,17
2,-0.608584,5.099506,2.205801,17
3,-11.358136,1.001592,-3.626971,17
4,0.973342,2.857534,5.820042,17
5,-7.636008,2.714807,-2.381496,17
6,-0.840795,6.072608,-0.739903,17
7,-5.605536,4.230971,1.174613,17
8,-2.886491,6.574297,0.532625,17
9,0.401929,0.330127,6.234346,17


Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color. **(P3-2: 10 points)**

In [180]:
fig = px.scatter_3d(df_pca, x='PC1', y='PC2', z='PC3', color='Label', symbol='Label', hover_data=['Label'])
fig.show()

# Sec 4: K-Means

Using K-means, we want to perform clustering on the 3D samples. Consider 4 clusters. After running the K-means algorithm on the data, determine to which cluster each sample belongs. Then, use `plotly` to plot the 3D samples colored with respect to their clusters. **(P4-1: 15 points)**

In [181]:
class KMeans:
    """
    Fits it on data, then uses predict to get cluster labels.
    """

    def __init__(self, n_clusters=8):
        """
        YOUR CODE (IF NECESSARY)
        """
        self.n_clusters = n_clusters

    def fit(self, X):
        """Fits the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = x.shape
        """
        YOUR CODE
        """
        self.centroids = X[np.random.choice(X.shape[0], self.n_clusters, replace=False)]
        while True:
            distances = np.sqrt(((X - self.centroids[:, np.newaxis])**2).sum(axis=2))
            labels = distances.argmin(axis=0)
            new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(self.n_clusters)])
            if np.allclose(new_centroids, self.centroids):
                break
            self.centroids = new_centroids

        return labels

    def predict(self, X):
        """ Predicts the cluster label
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted cluster label
        """

        y_pred = []
        """
        YOUR CODE
        """
        distances = np.sqrt(((X - self.centroids[:, np.newaxis])**2).sum(axis=2))
        y_pred = argmin(axis=0)
        return y_pred

In [182]:
kmeans_model = KMeans(4)
kmeans_labels = kmeans_model.fit(pca_result)
kmeans_labels

array([1, 3, 1, 2, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3,
       3, 2, 2, 2, 2, 2, 1, 1, 2, 3, 2, 1, 3, 2, 1, 2, 0, 3])

In [183]:
df_kmeans = pd.DataFrame({'PC1': pca_result[:, 0], 'PC2': pca_result[:, 1], 'PC3': pca_result[:, 2], 'Label': kmeans_labels})
df_kmeans

,PC1,PC2,PC3,Label
0,-1.373084,6.218771,1.209337,1
1,5.937131,1.932532,5.918058,3
2,-0.608584,5.099506,2.205801,1
3,-11.358136,1.001592,-3.626971,2
4,0.973342,2.857534,5.820042,1
5,-7.636008,2.714807,-2.381496,2
6,-0.840795,6.072608,-0.739903,1
7,-5.605536,4.230971,1.174613,1
8,-2.886491,6.574297,0.532625,1
9,0.401929,0.330127,6.234346,1


In [184]:
fig = px.scatter_3d(df_kmeans, x='PC1', y='PC2', z='PC3', color='Label', symbol='Label', hover_data=['Label'])
fig.show()

Measure the quality of the clustering using "silhoutte score" **(P4-2: 5 points)**

Note: You can use `sklearn` or other packages to calculate the score.

In [185]:
from sklearn.metrics import silhouette_score

In [186]:
silhouette_score = silhouette_score(pca_result, kmeans_labels)
silhouette_score

0.3519694605133452

Compare the result of the clustering with the original labels. To be more precise, in this part, you should clearly answer the following questions with details:<br>

- Does each cluster represent a true label?
- What is your conclusion?

**(P4-3: 10 points)**

In [187]:
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score

ari = adjusted_rand_score(filtered_labels, kmeans_labels)
nmi = normalized_mutual_info_score(filtered_labels, kmeans_labels)

print(f"Adjusted Rand Index: {ari:.2f}")
print(f"Normalized Mutual Information: {nmi:.2f}")

Adjusted Rand Index: 0.34
Normalized Mutual Information: 0.43


In [188]:
df_labels = pd.DataFrame({'Cluster': kmeans_labels, 'Label': filtered_labels})
cross_tab = pd.crosstab(df_labels['Cluster'], df_labels['Label'])
cross_tab

Label,2,17,33,37
Cluster,,,,
0,0,0,10,1
1,2,7,0,2
2,5,2,0,4
3,3,1,0,3


As we know, k-means clusterring, cluster data due to their distances from center of each cluster. By looking to the first plot we can see that the data for labels 37 and 17 are not close to each other which can not be captured by k-means as it can be seen in the cross tab. but labels like 33 which their samples are close together can easily be captured. Overall according to cross tab, Normalized mutal information, and adjusted rand index clusters can't present labels in this data.

# Sec 5: Neural Net (Optional with Bonus Points)

Reshape the images so that they become of shape `64x64` **(P5-1: 5 points)**

In [189]:
file_names = ['content/img-data/' + str(i) + '.jpg' for i in range(400)]
images_r = []
for file_name in file_names:
    with ZipFile( "images.zip", "r") as zip_ref:
        with zip_ref.open(file_name) as image_file:
            img = Image.open(image_file)
            img = img.resize((64, 64))
            img = np.array(img)
            images_r.append(img)
images_r = np.array(images_r)
images_r.shape

(400, 64, 64)

In [190]:
x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(images_r, y, test_size=0.1, stratify = y, random_state=1)
x_train_r, x_val_r, y_train_r, y_val_r = train_test_split(x_train_r, y_train_r, test_size=0.11, stratify = y_train_r, random_state=1)

Use different types of augmentation techniques (such as Random Flipping, Random Rotation, etc.) to increase the size of the training dataset. **(P5-2: 10 points)**

In [191]:
from keras.preprocessing.image import ImageDataGenerator

In [192]:
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
)

x_train_r = np.expand_dims(x_train_r, axis=-1)
datagen.fit(x_train_r)

batch_size = 32
x_augmented = []

for i in range(10):
    batch = x_train_r[i*batch_size:(i+1)*batch_size]
    augmented_batch = datagen.flow(batch, batch_size=batch_size, shuffle=False)
    x_augmented.append(augmented_batch[0])

x_augmented = np.squeeze(x_augmented, axis=-1)
x_augmented = np.reshape(x_augmented, (x_augmented.shape[0]*x_augmented.shape[1], x_augmented.shape[2], x_augmented.shape[3]))
x_augmented = np.expand_dims(x_augmented, axis=-1)

x_train_r = np.concatenate([x_train_r, x_augmented])
y_train_r = np.concatenate([y_train_r, y_train_r])

Using the augmented training set, train a neural network consisting of 2D convolutional layers. The exact architecture of the network is arbitrary. However, the last layer should have the "softmax" function as its activation. Also, using "max pooling" layers after convolutional ones is advised.

Note: Don't get frustrated if your network does not appear to be a good one. Give it some time to be trained. For instance, you might need to train it for 100 epochs. Make sure to try that out before quitting!

**(P5-3: 20 points)**

In [193]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, (2, 2), input_shape=(64, 64, 1)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (2, 2)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (2, 2)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

In [194]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='rmsprop',
              metrics=['accuracy'])

In [195]:
history = model.fit(np.array(x_train_r), y_train_r, epochs=10, validation_data=(x_val_r, y_val_r))

Epoch 1/10
20/20 [==============================] - 3s 140ms/step - loss: -1834225.6250 - accuracy: 0.0250 - val_loss: -8223101.0000 - val_accuracy: 0.0250
Epoch 2/10
20/20 [==============================] - 3s 135ms/step - loss: -30848106.0000 - accuracy: 0.0250 - val_loss: -71906464.0000 - val_accuracy: 0.0250
Epoch 3/10
20/20 [==============================] - 2s 83ms/step - loss: -154014592.0000 - accuracy: 0.0250 - val_loss: -279053344.0000 - val_accuracy: 0.0250
Epoch 4/10
20/20 [==============================] - 2s 97ms/step - loss: -471152992.0000 - accuracy: 0.0250 - val_loss: -744136320.0000 - val_accuracy: 0.0250
Epoch 5/10
20/20 [==============================] - 2s 76ms/step - loss: -1106149120.0000 - accuracy: 0.0250 - val_loss: -1608061696.0000 - val_accuracy: 0.0250
Epoch 6/10
20/20 [==============================] - 1s 74ms/step - loss: -2255572736.0000 - accuracy: 0.0250 - val_loss: -3113979648.0000 - val_accuracy: 0.0250
Epoch 7/10
20/20 [============================

Measure the accuracy of the network on the validation and testing datasets **(P5-4: 5 points)**

In [196]:
print('Validation')
val_loss, val_accuracy = model.evaluate(x_val_r, y_val_r)
print('Test')
test_loss, test_accuracy = model.evaluate(x_test_r, y_test_r)

Validation
2/2 [==============================] - 0s 14ms/step - loss: -20761120768.0000 - accuracy: 0.0250
Test
2/2 [==============================] - 0s 14ms/step - loss: -20791128064.0000 - accuracy: 0.0250
